## TRATANDO DADOS

1. Importação
2. Seleção dos Metais
3. Featurização (com ela, add colunas que correspondem a propriedades no nosso dataframe)
4. Retirada de dados faltantes
5. Arquivo picle;

Observação: redução do número de linhas

In [1]:
import pandas as pd  # biblioteca que permite análise e manipulação de dados
from matminer.datasets import (
    load_dataset,
)  # biblioteca que reúne dados de materiais e oferece diferentes ferramentas para manipulá-los
from matminer.featurizers.conversions import (
    StrToComposition,
    StructureToComposition
)  # composição estequiométrica
from matminer.featurizers.composition import (
    ElementProperty,
    ElementFraction,
    BandCenter,
)  # propriedades dos elementos
from matminer.featurizers.structure import DensityFeatures


colunas = pd.read_csv("Dados.csv")
colunas

/home/guilherme/anaconda3/envs/IP_venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


FileNotFoundError: [Errno 2] No such file or directory: 'Dados.csv'

In [2]:
datasets = [load_dataset(dataset) for dataset in colunas["0"]]


KeyboardInterrupt: 

In [ ]:
coluna_dadaset1 = list(colunas[["1", "2"]].iloc[0]) + ["formula", "structure"]
coluna_dadaset2 = [colunas["1"].iloc[1], "formula"]


In [ ]:
datasets[0] = datasets[0][coluna_dadaset1]
datasets[1] = datasets[1][coluna_dadaset2]


In [ ]:
dados_juntos = pd.merge(datasets[0], datasets[1], on="formula")
len(set(dados_juntos["formula"]))


36

In [ ]:
# ferramentas oferecidas pelo próprio matminer

# Carregando o dataset
df = datasets[0]

stc = StrToComposition()
df = stc.featurize_dataframe(df, "formula", pbar=False)

# Calculando as propriedades dos elementos
ep_feat = ElementProperty(
    "pymatgen", ["melting_point", "thermal_conductivity"], ["mean"]
)
ep_feat.set_n_jobs(7)
df = ep_feat.featurize_dataframe(df, col_id="composition")

# Selecionando as colunas que contém informações sobre condutividade térmica


ElementProperty: 100%|██████████| 8924/8924 [00:49<00:00, 179.54it/s]


In [ ]:
df = df.dropna(
    axis=1
)  # retirando as linhas que possuirem algum dado faltante (NaN, Not a Number)


In [ ]:
densityf = DensityFeatures()

densityf.feature_labels()  # utilizado para mostrar os dados disponiveis com esse metodo.


['density', 'vpa', 'packing fraction']

> vpa = volume per atom = estrutura dos materiais; calculado pela divisão da célula unitária pelo número de átomos nela

> packing fraction: measura of how efficiently space is filled by objects comprising a packing

In [ ]:
df = densityf.featurize_dataframe(df, "structure", True, True)


DensityFeatures: 100%|██████████| 8924/8924 [00:53<00:00, 165.70it/s]



In [ ]:
df = df.rename(columns={"PymatgenData mean melting_point": "melting_point"})
df = df.rename(
    columns={"PymatgenData mean thermal_conductivity": "thermal_conductivity"}
)


> Materiais (linhas) e as propriedades que conseguimos obter (colunas)

In [ ]:
df.describe()


,pf_n,pf_p,melting_point,thermal_conductivity,density,vpa,packing fraction
count,8.924000e+03,8924.000000,8924.000000,8924.000000,8889.000000,8889.000000,8889.000000
mean,6.662679e-01,0.299013,525.742521,31.202593,4.055432,17.817603,0.446406
std,4.668595e-01,0.434866,324.135984,31.960225,1.778242,8.324262,0.223698
min,8.930000e-07,0.000015,14.010000,0.008900,0.038513,5.571606,0.001506
25%,2.670000e-01,0.064100,312.013750,9.514377,2.764440,12.319221,0.285791
50%,7.100000e-01,0.166000,480.092222,22.443271,3.644737,14.781777,0.429114
75%,9.360000e-01,0.373000,664.696364,42.322361,5.069934,20.782544,0.575488
max,6.710000e+00,7.290000,3800.000000,300.006458,13.869557,73.999192,1.436588


In [ ]:
quantidade = len(set(df["formula"]))

print(f"Com esses dados temos {quantidade} moleculas distintas.")


Com esses dados temos 7380 moleculas distintas.


In [ ]:
df = load_dataset("matbench_expt_is_metal")
df = df[df["is_metal"] == True]
# ferramentas oferecidas pelo próprio matminer
df = df.rename(columns={"composition": "formula"})
# Carregando o dataset

stc = StrToComposition()
df = stc.featurize_dataframe(df, "formula")

# Calculando as propriedades dos elementos
ep_feat = ElementProperty(
    "pymatgen", ["melting_point", "thermal_conductivity"], ["mean"]
)
df = ep_feat.featurize_dataframe(df, col_id="composition")


frac = ElementFraction()
df = frac.featurize_dataframe(df, "composition")

BC = BandCenter()
df = BC.featurize_dataframe(df, "composition", True)

df.reset_index(inplace=True, drop=True)

df = df.dropna(axis=0)

df

ElementFraction: 100%|██████████| 2451/2451 [00:12<00:00, 194.63it/s]

BandCenter:  57%|█████▋    | 1409/2451 [00:12<00:08, 116.88it/s]


KeyboardInterrupt: 

In [ ]:
df.to_csv("Dados_teste.csv")

In [3]:
df = load_dataset("matbench_mp_is_metal")
df = df[df["is_metal"] == True]
# ferramentas oferecidas pelo próprio matminer

stc = StructureToComposition()
df = stc.featurize_dataframe(df, "structure")

df = df.dropna(axis=0)
# Calculando as propriedades dos elementos
ep_feat = ElementProperty(
    "pymatgen", ["melting_point", "thermal_conductivity"], ["mean"]
)
df = ep_feat.featurize_dataframe(df, col_id="composition")

df = df.dropna(axis=0)


frac = ElementFraction()
df = frac.featurize_dataframe(df, "composition")

df = df.dropna(axis=0)

BC = BandCenter()
df = BC.featurize_dataframe(df, "composition", True)
df = df.dropna(axis=0)

denst = DensityFeatures()
df = denst.featurize_dataframe(df, "structure")

df = df.dropna(axis=0)

df = df.rename(columns={"PymatgenData mean melting_point": "melting_point"})
df = df.rename(
    columns={"PymatgenData mean thermal_conductivity": "thermal_conductivity"}
)

df["formula"] = [item.reduced_formula for item in df["composition"]]

df = df.drop_duplicates(subset=["formula"])

df.reset_index(inplace=True, drop=True)

df.to_pickle("Dados")

StructureToComposition: 100%|██████████| 46151/46151 [01:44<00:00, 443.48it/s]

ElementFraction: 100%|██████████| 46151/46151 [03:53<00:00, 197.35it/s]

DensityFeatures:  86%|████████▌ | 37459/43715 [03:20<00:34, 183.80it/s]